### Created on 2025
### @author: S.W

## 텍스트 전처리 및 벡터화 실습 코드

### 라이브리리 불러오기

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import re
import math
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 라이브러리 설치 명령어 (주석 제거 후 실행)
"""
pip install nltk scikit-learn gensim transformers torch
python -m nltk.downloader punkt stopwords
"""

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec, FastText
from gensim.models.keyedvectors import KeyedVectors
import gensim.downloader as api

# 샘플 데이터
sample_texts = [
    "Natural language processing is a fascinating field of artificial intelligence.",
    "Machine learning algorithms can process and analyze large amounts of text data.",
    "Deep learning models have revolutionized natural language understanding.",
    "Text preprocessing is an essential step in NLP pipeline.",
    "Word embeddings capture semantic relationships between words.",
    "Transformers have become the dominant architecture in modern NLP.",
    "BERT and GPT models achieve state-of-the-art results in many tasks.",
    "Text classification and sentiment analysis are common NLP applications."
]

print("=== 텍스트 전처리 및 벡터화 실습 ===\n")
print("샘플 텍스트:")
for i, text in enumerate(sample_texts, 1):
    print(f"{i}. {text}")

=== 텍스트 전처리 및 벡터화 실습 ===

샘플 텍스트:
1. Natural language processing is a fascinating field of artificial intelligence.
2. Machine learning algorithms can process and analyze large amounts of text data.
3. Deep learning models have revolutionized natural language understanding.
4. Text preprocessing is an essential step in NLP pipeline.
5. Word embeddings capture semantic relationships between words.
6. Transformers have become the dominant architecture in modern NLP.
7. BERT and GPT models achieve state-of-the-art results in many tasks.
8. Text classification and sentiment analysis are common NLP applications.


### 1. 기본 텍스트 전처리

In [3]:
def basic_preprocessing(text):
    """
    기본적인 텍스트 전처리 함수
    
    Args:
        text (str): 원본 텍스트
    
    Returns:
        list: 전처리된 토큰 리스트
    """
    
    # 1단계: 소문자 변환 (대소문자 통일로 일관성 확보)
    # 예: "Natural" -> "natural"
    text = text.lower()
    print(f"    1단계 - 소문자 변환: {text}")
    
    # 2단계: 구두점 및 특수문자 제거
    # 정규표현식 [^\w\s]: 문자, 숫자, 공백이 아닌 모든 것을 제거
    # 예: "natural!" -> "natural"
    text = re.sub(r'[^\w\s]', '', text)
    print(f"    2단계 - 구두점 제거: {text}")
    
    # 3단계: 토큰화 (문장을 개별 단어로 분리)
    # word_tokenize: NLTK의 토큰화 함수 사용
    tokens = word_tokenize(text)
    print(f"    3단계 - 토큰화: {tokens}")
    
    # 4단계: 불용어 제거 (의미가 적은 일반적인 단어 제거)
    # 불용어 예: the, is, a, an, and, or, but 등
    try:
        # NLTK 영어 불용어 사전 로드
        stop_words = set(stopwords.words('english'))
        print(f"    불용어 예시: {list(stop_words)[:10]}...")
        
        # 불용어가 아닌 토큰만 유지
        filtered_tokens = []
        for token in tokens:
            if token not in stop_words:
                filtered_tokens.append(token)
            else:
                print(f"    제거된 불용어: '{token}'")
        
        tokens = filtered_tokens
        
    except LookupError:
        # NLTK 불용어 데이터가 없는 경우 대안 처리
        print("    NLTK stopwords not downloaded. Using basic preprocessing.")
        print("    대안: 길이가 2 이하인 단어 제거")
        tokens = [token for token in tokens if len(token) > 2]
    
    print(f"    4단계 - 불용어 제거 결과: {tokens}")
    
    return tokens

print("\n" + "="*50)
print("1. 기본 텍스트 전처리")
print("="*50)

# 각 샘플 텍스트에 대해 전처리 수행
preprocessed_texts = []  # 전처리된 결과를 저장할 리스트

print("전처리 과정을 단계별로 확인해보겠습니다:\n")

for i, text in enumerate(sample_texts):
    print(f"📝 문서 {i+1} 전처리:")
    print(f"원본: {text}")
    
    # 전처리 함수 호출 (각 단계별 진행상황이 출력됨)
    tokens = basic_preprocessing(text)
    
    # 결과를 리스트에 저장
    preprocessed_texts.append(tokens)
    
    print(f"✅ 최종 결과: {tokens}")
    print(f"토큰 개수: {len(tokens)}개")
    print("-" * 70 + "\n")


1. 기본 텍스트 전처리
전처리 과정을 단계별로 확인해보겠습니다:

📝 문서 1 전처리:
원본: Natural language processing is a fascinating field of artificial intelligence.
    1단계 - 소문자 변환: natural language processing is a fascinating field of artificial intelligence.
    2단계 - 구두점 제거: natural language processing is a fascinating field of artificial intelligence
    3단계 - 토큰화: ['natural', 'language', 'processing', 'is', 'a', 'fascinating', 'field', 'of', 'artificial', 'intelligence']
    불용어 예시: ['when', 'his', 'had', 'wasn', 'don', 'will', "he's", 'o', 'where', 'other']...
    제거된 불용어: 'is'
    제거된 불용어: 'a'
    제거된 불용어: 'of'
    4단계 - 불용어 제거 결과: ['natural', 'language', 'processing', 'fascinating', 'field', 'artificial', 'intelligence']
✅ 최종 결과: ['natural', 'language', 'processing', 'fascinating', 'field', 'artificial', 'intelligence']
토큰 개수: 7개
----------------------------------------------------------------------

📝 문서 2 전처리:
원본: Machine learning algorithms can process and analyze large amounts of text data.
    1단계 - 소문

-----

### 2. Bag of Words (BoW)

Bag of Words (BoW) 개념:
- 문서를 단어의 집합(가방)으로 표현
- 단어의 순서는 무시하고, 출현 빈도만 고려
- 각 문서를 고정된 크기의 벡터로 변환
- 벡터의 각 차원은 특정 단어의 출현 횟수를 나타냄

In [4]:
print("\n" + "="*50)
print("2. Bag of Words (BoW)")
print("="*50)

def create_bow_manual(texts):
    """
    수동으로 BoW 벡터 생성하기
    
    Args:
        texts (list): 전처리된 토큰 리스트들
    
    Returns:
        tuple: (BoW 행렬, 어휘 사전)
    """
    
    print(" 1단계: 전체 문서에서 고유 단어 수집")
    # 모든 문서의 모든 단어를 하나의 집합으로 수집
    all_words = set()
    for i, tokens in enumerate(texts):
        print(f"  문서 {i+1}의 단어들: {tokens}")
        all_words.update(tokens)  # 집합에 단어들 추가
    
    # 어휘 사전 생성 (알파벳 순으로 정렬)
    vocab = sorted(list(all_words))
    print(f"\n📖 어휘 사전 생성 완료!")
    print(f"  - 총 고유 단어 수: {len(vocab)}개")
    print(f"  - 어휘 사전 (처음 15개): {vocab[:15]}")
    if len(vocab) > 15:
        print(f"  - ... (나머지 {len(vocab)-15}개)")
    
    print(f"\n 2단계: 각 문서를 BoW 벡터로 변환")
    # 각 문서에 대해 BoW 벡터 생성
    bow_vectors = []
    
    for doc_idx, tokens in enumerate(texts):
        print(f"\n  📄 문서 {doc_idx+1} 처리 중...")
        print(f"    토큰: {tokens}")
        
        # 각 어휘에 대해 해당 문서에서의 출현 횟수 계산
        vector = []
        word_counts = {}
        
        for word in vocab:
            count = tokens.count(word)  # 해당 단어의 출현 횟수
            vector.append(count)
            if count > 0:  # 출현한 단어만 기록
                word_counts[word] = count
        
        bow_vectors.append(vector)
        
        # 출현한 단어들만 출력
        print(f"    출현한 단어와 빈도: {word_counts}")
        print(f"    BoW 벡터 길이: {len(vector)}")
        print(f"    0이 아닌 값의 개수: {sum(1 for x in vector if x > 0)}")
    
    return np.array(bow_vectors), vocab

print("=" * 70)
bow_manual, vocab_manual = create_bow_manual(preprocessed_texts)

print(f"\n BoW 행렬 생성 완료!")
print(f" 행렬 크기: {bow_manual.shape} (문서 수 × 어휘 크기)")
print(f" 첫 번째 문서의 BoW 벡터 (처음 10차원): {bow_manual[0][:10]}")

# 희소성 계산
total_elements = bow_manual.shape[0] * bow_manual.shape[1]
zero_elements = np.sum(bow_manual == 0)
sparsity = (zero_elements / total_elements) * 100
print(f" 희소성(Sparsity): {sparsity:.1f}% (대부분의 값이 0)")


2. Bag of Words (BoW)
 1단계: 전체 문서에서 고유 단어 수집
  문서 1의 단어들: ['natural', 'language', 'processing', 'fascinating', 'field', 'artificial', 'intelligence']
  문서 2의 단어들: ['machine', 'learning', 'algorithms', 'process', 'analyze', 'large', 'amounts', 'text', 'data']
  문서 3의 단어들: ['deep', 'learning', 'models', 'revolutionized', 'natural', 'language', 'understanding']
  문서 4의 단어들: ['text', 'preprocessing', 'essential', 'step', 'nlp', 'pipeline']
  문서 5의 단어들: ['word', 'embeddings', 'capture', 'semantic', 'relationships', 'words']
  문서 6의 단어들: ['transformers', 'become', 'dominant', 'architecture', 'modern', 'nlp']
  문서 7의 단어들: ['bert', 'gpt', 'models', 'achieve', 'stateoftheart', 'results', 'many', 'tasks']
  문서 8의 단어들: ['text', 'classification', 'sentiment', 'analysis', 'common', 'nlp', 'applications']

📖 어휘 사전 생성 완료!
  - 총 고유 단어 수: 48개
  - 어휘 사전 (처음 15개): ['achieve', 'algorithms', 'amounts', 'analysis', 'analyze', 'applications', 'architecture', 'artificial', 'become', 'bert', 'capture', 'class

**실제 프로젝트에서는 Scikit-learn의 CountVectorizer를 주로 사용합니다.<br>
더 효율적이고 다양한 옵션을 제공합니다.**

In [5]:
print("\n" + "="*50)
print("📚 Scikit-learn을 사용한 BoW")
print("="*50)

# Scikit-learn CountVectorizer 사용
vectorizer_bow = CountVectorizer(
    tokenizer=lambda x: x,  # 이미 토큰화된 데이터 사용
    lowercase=False         # 이미 소문자 변환 완료
)

bow_sklearn = vectorizer_bow.fit_transform(preprocessed_texts)

print(f"🔧 Scikit-learn BoW 설정:")
print(f"  - 어휘 크기: {len(vectorizer_bow.vocabulary_)}")
print(f"  - 행렬 크기: {bow_sklearn.shape}")
print(f"  - 행렬 타입: {type(bow_sklearn)} (메모리 효율적인 희소 행렬)")

# 어휘 사전 비교
sklearn_vocab = sorted(vectorizer_bow.vocabulary_.keys())
print(f"\n🔍 어휘 사전 비교:")
print(f"  - 수동 구현: {len(vocab_manual)}개 단어")
print(f"  - Scikit-learn: {len(sklearn_vocab)}개 단어")
print(f"  - 동일한 결과: {vocab_manual == sklearn_vocab}")

# 희소 행렬을 밀집 행렬로 변환하여 비교
bow_sklearn_dense = bow_sklearn.toarray()
print(f"\n📊 결과 비교:")
print(f"  - 수동 구현 첫 문서: {bow_manual[0][:5]}...")
print(f"  - Scikit-learn 첫 문서: {bow_sklearn_dense[0][:5]}...")
print(f"  - 결과 일치: {np.array_equal(bow_manual, bow_sklearn_dense)}")


📚 Scikit-learn을 사용한 BoW
🔧 Scikit-learn BoW 설정:
  - 어휘 크기: 48
  - 행렬 크기: (8, 48)
  - 행렬 타입: <class 'scipy.sparse._csr.csr_matrix'> (메모리 효율적인 희소 행렬)

🔍 어휘 사전 비교:
  - 수동 구현: 48개 단어
  - Scikit-learn: 48개 단어
  - 동일한 결과: True

📊 결과 비교:
  - 수동 구현 첫 문서: [0 0 0 0 0]...
  - Scikit-learn 첫 문서: [0 0 0 0 0]...
  - 결과 일치: True


-----

### 3. N-gram
N-gram 개념:
- N개의 연속된 단어를 하나의 단위로 취급
- 단어 간의 순서와 맥락 정보를 어느 정도 보존
- 1-gram(unigram): 개별 단어
- 2-gram(bigram): 두 개의 연속된 단어
- 3-gram(trigram): 세 개의 연속된 단어

In [6]:
print("\n" + "="*50)
print("3. N-gram")
print("="*50)

print("""
📚 N-gram 개념:
- N개의 연속된 단어를 하나의 단위로 취급
- 단어 간의 순서와 맥락 정보를 어느 정도 보존
- 1-gram(unigram): 개별 단어
- 2-gram(bigram): 두 개의 연속된 단어
- 3-gram(trigram): 세 개의 연속된 단어
""")

def create_ngrams(tokens, n):
    """
    N-gram 생성 함수
    
    Args:
        tokens (list): 토큰 리스트
        n (int): N-gram의 N 값
    
    Returns:
        list: N-gram 리스트
    """
    print(f"🔍 {n}-gram 생성 과정:")
    print(f"  입력 토큰: {tokens}")
    print(f"  토큰 개수: {len(tokens)}")
    
    ngrams = []
    
    # 슬라이딩 윈도우 방식으로 N-gram 생성
    for i in range(len(tokens) - n + 1):
        # i번째부터 i+n번째까지의 토큰들을 결합
        ngram_tokens = tokens[i:i+n]
        ngram = ' '.join(ngram_tokens)
        ngrams.append(ngram)
        print(f"  위치 {i}: {ngram_tokens} → '{ngram}'")
    
    print(f"  생성된 {n}-gram 개수: {len(ngrams)}")
    return ngrams

# 첫 번째 문서로 N-gram 예시 생성
sample_tokens = preprocessed_texts[0]
print(f"🎯 샘플 문서 분석:")
print(f"원본 문서: {sample_texts[0]}")
print(f"전처리된 토큰: {sample_tokens}")

print("\n" + "="*40)
print("📝 1-gram (Unigram) 생성")
print("="*40)
unigrams = create_ngrams(sample_tokens, 1)

print("\n" + "="*40)
print("📝 2-gram (Bigram) 생성")
print("="*40)
bigrams = create_ngrams(sample_tokens, 2)

print("\n" + "="*40)
print("📝 3-gram (Trigram) 생성")
print("="*40)
trigrams = create_ngrams(sample_tokens, 3)

# N-gram 특성 분석
print(f"\n📊 N-gram 분석 결과:")
print(f"  원본 토큰 수: {len(sample_tokens)}")
print(f"  1-gram 수: {len(unigrams)} (= 토큰 수)")
print(f"  2-gram 수: {len(bigrams)} (= 토큰 수 - 1)")
print(f"  3-gram 수: {len(trigrams)} (= 토큰 수 - 2)")

print(f"\n🔍 생성된 N-gram 예시:")
print(f"  1-gram: {unigrams}")
print(f"  2-gram: {bigrams}")
print(f"  3-gram: {trigrams}")


3. N-gram

📚 N-gram 개념:
- N개의 연속된 단어를 하나의 단위로 취급
- 단어 간의 순서와 맥락 정보를 어느 정도 보존
- 1-gram(unigram): 개별 단어
- 2-gram(bigram): 두 개의 연속된 단어
- 3-gram(trigram): 세 개의 연속된 단어

🎯 샘플 문서 분석:
원본 문서: Natural language processing is a fascinating field of artificial intelligence.
전처리된 토큰: ['natural', 'language', 'processing', 'fascinating', 'field', 'artificial', 'intelligence']

📝 1-gram (Unigram) 생성
🔍 1-gram 생성 과정:
  입력 토큰: ['natural', 'language', 'processing', 'fascinating', 'field', 'artificial', 'intelligence']
  토큰 개수: 7
  위치 0: ['natural'] → 'natural'
  위치 1: ['language'] → 'language'
  위치 2: ['processing'] → 'processing'
  위치 3: ['fascinating'] → 'fascinating'
  위치 4: ['field'] → 'field'
  위치 5: ['artificial'] → 'artificial'
  위치 6: ['intelligence'] → 'intelligence'
  생성된 1-gram 개수: 7

📝 2-gram (Bigram) 생성
🔍 2-gram 생성 과정:
  입력 토큰: ['natural', 'language', 'processing', 'fascinating', 'field', 'artificial', 'intelligence']
  토큰 개수: 7
  위치 0: ['natural', 'language'] → 'natural language'
  위치 1: ['l

#### Scikit-learn을 사용한 N-gram
N-gram 벡터화 설정:
- ngram_range=(1, 3): 1-gram부터 3-gram까지 모두 포함
- max_features=50: 최대 50개의 특성만 선택 (빈도가 높은 순)
<br>

**실제 프로젝트에서는 Scikit-learn의 CountVectorizer에서 ngram_range 매개변수를 사용하여 N-gram을 생성합니다.**

In [7]:
vectorizer_ngram = CountVectorizer(
    ngram_range=(1, 3),    # 1-gram부터 3-gram까지
    max_features=50,       # 최대 특성 수 제한 (차원 폭발 방지)
    stop_words='english'   # 영어 불용어 자동 제거
)

# 원본 텍스트에서 직접 N-gram 생성
ngram_matrix = vectorizer_ngram.fit_transform(sample_texts)

print(f"\n📊 N-gram 행렬 정보:")
print(f"  - 행렬 크기: {ngram_matrix.shape}")
print(f"  - 문서 수: {ngram_matrix.shape[0]}")
print(f"  - N-gram 특성 수: {ngram_matrix.shape[1]}")


📊 N-gram 행렬 정보:
  - 행렬 크기: (8, 50)
  - 문서 수: 8
  - N-gram 특성 수: 50


In [8]:
# 생성된 N-gram 특성 확인
feature_names = vectorizer_ngram.get_feature_names_out()
print(f"생성된 N-gram 특성 예시 (처음 20개):")
for i, feature in enumerate(feature_names[:20]):
    feature_type = "1-gram" if ' ' not in feature else f"{len(feature.split())}-gram"
    print(f"  {i+1:2d}. '{feature}' ({feature_type})")

if len(feature_names) > 20:
    print(f"  ... (나머지 {len(feature_names)-20}개)")

생성된 N-gram 특성 예시 (처음 20개):
   1. 'achieve' (1-gram)
   2. 'language' (1-gram)
   3. 'learning' (1-gram)
   4. 'learning algorithms process' (3-gram)
   5. 'learning models' (2-gram)
   6. 'learning models revolutionized' (3-gram)
   7. 'machine' (1-gram)
   8. 'machine learning' (2-gram)
   9. 'machine learning algorithms' (3-gram)
  10. 'models' (1-gram)
  11. 'models achieve' (2-gram)
  12. 'models achieve state' (3-gram)
  13. 'models revolutionized' (2-gram)
  14. 'models revolutionized natural' (3-gram)
  15. 'modern' (1-gram)
  16. 'modern nlp' (2-gram)
  17. 'natural' (1-gram)
  18. 'natural language' (2-gram)
  19. 'natural language processing' (3-gram)
  20. 'natural language understanding' (3-gram)
  ... (나머지 30개)


In [9]:
# 첫 번째 문서의 N-gram 분석
first_doc_vector = ngram_matrix[0].toarray().flatten()
non_zero_indices = np.where(first_doc_vector > 0)[0]

print(f"첫 번째 문서에서 출현한 N-gram:")
print(f"원본: {sample_texts[0]}")
for idx in non_zero_indices[:10]:  # 처음 10개만 출력
    feature = feature_names[idx]
    count = first_doc_vector[idx]
    feature_type = "1-gram" if ' ' not in feature else f"{len(feature.split())}-gram"
    print(f"  '{feature}' ({feature_type}): {count}회")

if len(non_zero_indices) > 10:
    print(f"  ... (나머지 {len(non_zero_indices)-10}개)")

첫 번째 문서에서 출현한 N-gram:
원본: Natural language processing is a fascinating field of artificial intelligence.
  'language' (1-gram): 1회
  'natural' (1-gram): 1회
  'natural language' (2-gram): 1회
  'natural language processing' (3-gram): 1회
  'processing' (1-gram): 1회
  'processing fascinating' (2-gram): 1회
  'processing fascinating field' (3-gram): 1회


-----

### 4. TF-IDF (Term Frequency-Inverse Document Frequency)
TF-IDF 개념:
- TF (Term Frequency): 특정 문서에서 단어의 출현 빈도
- IDF (Inverse Document Frequency): 전체 문서에서 단어의 희귀성
- TF-IDF = TF × IDF
- 목적: 자주 나오지만 특별한 의미가 없는 단어의 가중치를 낮춤
- 문서의 핵심 내용을 나타내는 단어에 높은 가중치 부여

In [10]:
# 첫 3개 문서로 예시 실행 (출력을 간결하게 하기 위해)
sample_docs = preprocessed_texts[:3]
print("\n🎯 TF-IDF 계산 예시 (첫 3개 문서)")
print("원본 문서들:")
for i in range(3):
    print(f"  문서 {i+1}:")
    print(f"    원본: {sample_texts[i]}")
    print(f"    토큰: {sample_docs[i]}")


🎯 TF-IDF 계산 예시 (첫 3개 문서)
원본 문서들:
  문서 1:
    원본: Natural language processing is a fascinating field of artificial intelligence.
    토큰: ['natural', 'language', 'processing', 'fascinating', 'field', 'artificial', 'intelligence']
  문서 2:
    원본: Machine learning algorithms can process and analyze large amounts of text data.
    토큰: ['machine', 'learning', 'algorithms', 'process', 'analyze', 'large', 'amounts', 'text', 'data']
  문서 3:
    원본: Deep learning models have revolutionized natural language understanding.
    토큰: ['deep', 'learning', 'models', 'revolutionized', 'natural', 'language', 'understanding']


In [11]:
print("\n" + "="*50)
print("4. TF-IDF (Term Frequency-Inverse Document Frequency)")
print("="*50)

def calculate_tf(tokens):
    """
    TF (Term Frequency) 계산하기
    TF = (특정 단어의 출현 횟수) / (문서의 총 단어 수)
    """
    print(f"📊 TF 계산 과정:")
    print(f"  입력 토큰: {tokens}")
    
    tf_dict = {}
    total_count = len(tokens)
    print(f"  총 단어 수: {total_count}")
    
    # 각 단어의 출현 횟수 계산
    word_counts = Counter(tokens)
    print(f"  단어별 출현 횟수: {dict(word_counts)}")
    
    # TF 값 계산 
    for token, count in word_counts.items():
        tf_value = count / total_count
        tf_dict[token] = tf_value
        print(f"  '{token}': {count}/{total_count} = {tf_value:.4f}")
    
    return tf_dict

def calculate_idf(documents):
    """
    IDF (Inverse Document Frequency) 계산하기
    IDF = log(전체 문서 수 / 해당 단어가 포함된 문서 수)
    """
    print(f"\n📊 IDF 계산 과정:")
    
    idf_dict = {}
    total_documents = len(documents)
    print(f"  전체 문서 수: {total_documents}")
    
    # 모든 고유 단어 수집
    all_words = set()
    for doc in documents:
        all_words.update(doc)
    
    print(f"  전체 고유 단어 수: {len(all_words)}")
    
    # 각 단어에 대해 IDF 계산
    for word in sorted(all_words):
        # 해당 단어가 포함된 문서 수 계산
        containing_docs = 0
        for doc in documents:
            if word in doc:
                containing_docs += 1
        
        # IDF 계산
        idf_value = math.log(total_documents / containing_docs)
        idf_dict[word] = idf_value
        
        print(f"  '{word}': log({total_documents}/{containing_docs}) = {idf_value:.4f}")
    
    return idf_dict

def calculate_tfidf_manual(documents):
    """
    TF-IDF 수동으로 계산하기
    """
    print("🔍 TF-IDF 계산 시작")
    print("="*50)
    
    # 1단계: IDF 계산 (전체 문서에 대해)
    idf = calculate_idf(documents)
    
    print(f"\n📋 IDF 값 요약:")
    sorted_idf = sorted(idf.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_idf[:10]:
        print(f"  '{word}': {score:.4f} (희귀한 단어일수록 높은 값)")
    
    # 2단계: 각 문서별 TF-IDF 계산
    tfidf_documents = []
    
    for doc_idx, doc in enumerate(documents):
        print(f"\n📄 문서 {doc_idx+1} TF-IDF 계산:")
        print(f"  토큰: {doc}")
        
        # TF 계산
        tf = calculate_tf(doc)
        
        # TF-IDF 계산
        tfidf_doc = {}
        print(f"\n  TF-IDF = TF × IDF 계산:")
        for word, tf_val in tf.items():
            tfidf_val = tf_val * idf[word]
            tfidf_doc[word] = tfidf_val
            print(f"    '{word}': {tf_val:.4f} × {idf[word]:.4f} = {tfidf_val:.4f}")
        
        tfidf_documents.append(tfidf_doc)
        
        # 해당 문서의 중요 단어 출력
        sorted_tfidf = sorted(tfidf_doc.items(), key=lambda x: x[1], reverse=True)
        print(f"  💡 문서 {doc_idx+1}의 핵심 단어 (TF-IDF 기준):")
        for word, score in sorted_tfidf[:3]:
            print(f"    '{word}': {score:.4f}")
    
    return tfidf_documents, idf

tfidf_manual, idf_scores = calculate_tfidf_manual(sample_docs)


4. TF-IDF (Term Frequency-Inverse Document Frequency)
🔍 TF-IDF 계산 시작

📊 IDF 계산 과정:
  전체 문서 수: 3
  전체 고유 단어 수: 20
  'algorithms': log(3/1) = 1.0986
  'amounts': log(3/1) = 1.0986
  'analyze': log(3/1) = 1.0986
  'artificial': log(3/1) = 1.0986
  'data': log(3/1) = 1.0986
  'deep': log(3/1) = 1.0986
  'fascinating': log(3/1) = 1.0986
  'field': log(3/1) = 1.0986
  'intelligence': log(3/1) = 1.0986
  'language': log(3/2) = 0.4055
  'large': log(3/1) = 1.0986
  'learning': log(3/2) = 0.4055
  'machine': log(3/1) = 1.0986
  'models': log(3/1) = 1.0986
  'natural': log(3/2) = 0.4055
  'process': log(3/1) = 1.0986
  'processing': log(3/1) = 1.0986
  'revolutionized': log(3/1) = 1.0986
  'text': log(3/1) = 1.0986
  'understanding': log(3/1) = 1.0986

📋 IDF 값 요약:
  'algorithms': 1.0986 (희귀한 단어일수록 높은 값)
  'amounts': 1.0986 (희귀한 단어일수록 높은 값)
  'analyze': 1.0986 (희귀한 단어일수록 높은 값)
  'artificial': 1.0986 (희귀한 단어일수록 높은 값)
  'data': 1.0986 (희귀한 단어일수록 높은 값)
  'deep': 1.0986 (희귀한 단어일수록 높은 값)
  'fascinati

In [12]:
print("\n" + "="*50)
print("📊 TF-IDF 결과 분석")
print("="*50)

# 각 문서별 상위 단어 분석
for doc_idx in range(len(tfidf_manual)):
    print(f"\n📄 문서 {doc_idx+1} 분석:")
    print(f"  원본: {sample_texts[doc_idx]}")
    
    tfidf_doc = tfidf_manual[doc_idx]
    sorted_tfidf = sorted(tfidf_doc.items(), key=lambda x: x[1], reverse=True)
    print(f"  TF-IDF 상위 단어:")
    for rank, (word, score) in enumerate(sorted_tfidf, 1):
        print(f"    {rank}. '{word}': {score:.4f}")


📊 TF-IDF 결과 분석

📄 문서 1 분석:
  원본: Natural language processing is a fascinating field of artificial intelligence.
  TF-IDF 상위 단어:
    1. 'processing': 0.1569
    2. 'fascinating': 0.1569
    3. 'field': 0.1569
    4. 'artificial': 0.1569
    5. 'intelligence': 0.1569
    6. 'natural': 0.0579
    7. 'language': 0.0579

📄 문서 2 분석:
  원본: Machine learning algorithms can process and analyze large amounts of text data.
  TF-IDF 상위 단어:
    1. 'machine': 0.1221
    2. 'algorithms': 0.1221
    3. 'process': 0.1221
    4. 'analyze': 0.1221
    5. 'large': 0.1221
    6. 'amounts': 0.1221
    7. 'text': 0.1221
    8. 'data': 0.1221
    9. 'learning': 0.0451

📄 문서 3 분석:
  원본: Deep learning models have revolutionized natural language understanding.
  TF-IDF 상위 단어:
    1. 'deep': 0.1569
    2. 'models': 0.1569
    3. 'revolutionized': 0.1569
    4. 'understanding': 0.1569
    5. 'learning': 0.0579
    6. 'natural': 0.0579
    7. 'language': 0.0579


#### Scikit-learn을 사용한 TF-IDF

**실제 프로젝트에서는 Scikit-learn의 TfidfVectorizer를 사용합니다.
더 효율적이고 다양한 옵션을 제공합니다.**

In [13]:
# 전처리된 토큰을 다시 문자열로 변환 (Scikit-learn이 문자열을 받기 때문)
preprocessed_strings = []
for tokens in preprocessed_texts:
    text_string = ' '.join(tokens)
    preprocessed_strings.append(text_string)

print("전처리된 문자열들:")
for i, text in enumerate(preprocessed_strings):
    print(f"  {i+1}. {text}")

전처리된 문자열들:
  1. natural language processing fascinating field artificial intelligence
  2. machine learning algorithms process analyze large amounts text data
  3. deep learning models revolutionized natural language understanding
  4. text preprocessing essential step nlp pipeline
  5. word embeddings capture semantic relationships words
  6. transformers become dominant architecture modern nlp
  7. bert gpt models achieve stateoftheart results many tasks
  8. text classification sentiment analysis common nlp applications


In [14]:
# Scikit-learn TfidfVectorizer 사용
tfidf_vectorizer = TfidfVectorizer(
    lowercase=False,          # 이미 소문자 변환 완료
    stop_words=None,          # 이미 불용어 제거 완료
    norm='l2',               # L2 정규화 
    use_idf=True,            # IDF 사용
    smooth_idf=True,         # 안정적인 계산을 위한 스무딩
    sublinear_tf=False       # 기본 TF 사용
)

tfidf_sklearn = tfidf_vectorizer.fit_transform(preprocessed_strings)

print(f"\n🔧 Scikit-learn TF-IDF 결과:")
print(f"  - 행렬 크기: {tfidf_sklearn.shape} (문서 수 × 단어 수)")
print(f"  - 어휘 크기: {len(tfidf_vectorizer.vocabulary_)}개")


🔧 Scikit-learn TF-IDF 결과:
  - 행렬 크기: (8, 48) (문서 수 × 단어 수)
  - 어휘 크기: 48개


In [15]:
# 어휘 확인
print(f"\n📖 전체 어휘:")
vocabulary = tfidf_vectorizer.vocabulary_
sorted_vocab = sorted(vocabulary.items(), key=lambda x: x[1])
for word, idx in sorted_vocab:
    print(f"  {idx}: '{word}'")


📖 전체 어휘:
  0: 'achieve'
  1: 'algorithms'
  2: 'amounts'
  3: 'analysis'
  4: 'analyze'
  5: 'applications'
  6: 'architecture'
  7: 'artificial'
  8: 'become'
  9: 'bert'
  10: 'capture'
  11: 'classification'
  12: 'common'
  13: 'data'
  14: 'deep'
  15: 'dominant'
  16: 'embeddings'
  17: 'essential'
  18: 'fascinating'
  19: 'field'
  20: 'gpt'
  21: 'intelligence'
  22: 'language'
  23: 'large'
  24: 'learning'
  25: 'machine'
  26: 'many'
  27: 'models'
  28: 'modern'
  29: 'natural'
  30: 'nlp'
  31: 'pipeline'
  32: 'preprocessing'
  33: 'process'
  34: 'processing'
  35: 'relationships'
  36: 'results'
  37: 'revolutionized'
  38: 'semantic'
  39: 'sentiment'
  40: 'stateoftheart'
  41: 'step'
  42: 'tasks'
  43: 'text'
  44: 'transformers'
  45: 'understanding'
  46: 'word'
  47: 'words'


In [16]:
# IDF 값 확인
feature_names = tfidf_vectorizer.get_feature_names_out()
idf_values = tfidf_vectorizer.idf_

print(f"\n📊 Scikit-learn IDF 값:")
for i, word in enumerate(feature_names):
    print(f"  '{word}': {idf_values[i]:.4f}")

# 각 문서의 TF-IDF 값 확인 (첫 3개 문서)
print(f"\n📈 각 문서의 TF-IDF 값:")
for doc_idx in range(min(3, tfidf_sklearn.shape[0])):
    doc_tfidf = tfidf_sklearn[doc_idx].toarray().flatten()
    
    print(f"\n문서 {doc_idx+1}:")
    print(f"  원본: {sample_texts[doc_idx]}")
    
    # 0이 아닌 값들만 출력
    word_scores = []
    for i, score in enumerate(doc_tfidf):
        if score > 0:
            word = feature_names[i]
            word_scores.append((word, score))
    
    # 점수 순으로 정렬
    word_scores.sort(key=lambda x: x[1], reverse=True)
    
    for rank, (word, score) in enumerate(word_scores, 1):
        print(f"  {rank}. '{word}': {score:.4f}")


📊 Scikit-learn IDF 값:
  'achieve': 2.5041
  'algorithms': 2.5041
  'amounts': 2.5041
  'analysis': 2.5041
  'analyze': 2.5041
  'applications': 2.5041
  'architecture': 2.5041
  'artificial': 2.5041
  'become': 2.5041
  'bert': 2.5041
  'capture': 2.5041
  'classification': 2.5041
  'common': 2.5041
  'data': 2.5041
  'deep': 2.5041
  'dominant': 2.5041
  'embeddings': 2.5041
  'essential': 2.5041
  'fascinating': 2.5041
  'field': 2.5041
  'gpt': 2.5041
  'intelligence': 2.5041
  'language': 2.0986
  'large': 2.5041
  'learning': 2.0986
  'machine': 2.5041
  'many': 2.5041
  'models': 2.0986
  'modern': 2.5041
  'natural': 2.0986
  'nlp': 1.8109
  'pipeline': 2.5041
  'preprocessing': 2.5041
  'process': 2.5041
  'processing': 2.5041
  'relationships': 2.5041
  'results': 2.5041
  'revolutionized': 2.5041
  'semantic': 2.5041
  'sentiment': 2.5041
  'stateoftheart': 2.5041
  'step': 2.5041
  'tasks': 2.5041
  'text': 1.8109
  'transformers': 2.5041
  'understanding': 2.5041
  'word':

-----

### 5. Word2Vec (Word to Vector)
Word2Vec 개념:
- 단어를 고정 크기의 실수 벡터로 변환하는 기술
- 비슷한 의미의 단어들이 벡터 공간에서 가까운 위치에 배치됨
- TF-IDF와 달리 단어 간 의미적 관계를 학습함
- 두 가지 모델: CBOW (주변 단어로 중심 단어 예측), Skip-gram (중심 단어로 주변 단어 예측)

Word2Vec의 핵심 아이디어:
- "비슷한 맥락에서 나타나는 단어들은 비슷한 의미를 가진다"
- 예: "강아지는 귀엽다", "고양이는 귀엽다" → '강아지'와 '고양이'는 유사한 벡터를 가짐

In [17]:
def show_preprocessing_result(preprocessed_texts):
    """전처리 결과를 다시 한번 확인"""
    print("🔍 Word2Vec 학습에 사용할 데이터 확인:")
    print("전처리된 문서들:")
    for i, tokens in enumerate(preprocessed_texts):
        print(f"  문서 {i+1}: {tokens}")
        print(f"    토큰 수: {len(tokens)}개")
    
    # 전체 어휘 통계
    all_words = []
    for tokens in preprocessed_texts:
        all_words.extend(tokens)
    
    word_counts = Counter(all_words)
    print(f"\n📊 전체 어휘 통계:")
    print(f"  총 토큰 수: {len(all_words)}개")
    print(f"  고유 단어 수: {len(word_counts)}개")
    print(f"  가장 빈번한 단어 5개:")
    for word, count in word_counts.most_common(5):
        print(f"    '{word}': {count}번")

show_preprocessing_result(preprocessed_texts)

🔍 Word2Vec 학습에 사용할 데이터 확인:
전처리된 문서들:
  문서 1: ['natural', 'language', 'processing', 'fascinating', 'field', 'artificial', 'intelligence']
    토큰 수: 7개
  문서 2: ['machine', 'learning', 'algorithms', 'process', 'analyze', 'large', 'amounts', 'text', 'data']
    토큰 수: 9개
  문서 3: ['deep', 'learning', 'models', 'revolutionized', 'natural', 'language', 'understanding']
    토큰 수: 7개
  문서 4: ['text', 'preprocessing', 'essential', 'step', 'nlp', 'pipeline']
    토큰 수: 6개
  문서 5: ['word', 'embeddings', 'capture', 'semantic', 'relationships', 'words']
    토큰 수: 6개
  문서 6: ['transformers', 'become', 'dominant', 'architecture', 'modern', 'nlp']
    토큰 수: 6개
  문서 7: ['bert', 'gpt', 'models', 'achieve', 'stateoftheart', 'results', 'many', 'tasks']
    토큰 수: 8개
  문서 8: ['text', 'classification', 'sentiment', 'analysis', 'common', 'nlp', 'applications']
    토큰 수: 7개

📊 전체 어휘 통계:
  총 토큰 수: 56개
  고유 단어 수: 48개
  가장 빈번한 단어 5개:
    'text': 3번
    'nlp': 3번
    'natural': 2번
    'language': 2번
    'learning': 2

Word2Vec 주요 파라미터 설명:
- vector_size=100   : 각 단어를 100차원 벡터로 표현
- window=5         : 중심 단어 기준 앞뒤 5개 단어까지 고려
- min_count=1      : 최소 1번 이상 나타난 단어만 학습 (데이터가 적어서 1로 설정)
- workers=4        : 4개 프로세스로 병렬 학습
- sg=0            : CBOW 모델 사용 (0=CBOW, 1=Skip-gram)
- epochs=10       : 전체 데이터를 10번 반복 학습
<br>

CBOW vs Skip-gram:
<br>

CBOW (Continuous Bag of Words):
- 주변 단어들로 중심 단어를 예측
- 예: '자연어 [?] 는 흥미롭다' → '처리'를 예측
- 빠른 학습, 빈번한 단어에 좋음

<br>

Skip-gram:
- 중심 단어로 주변 단어들을 예측
- 예: '처리' → '자연어', '는', '흥미롭다' 예측
- 정확한 벡터, 희귀 단어에 좋음

In [18]:
# Word2Vec 모델 학습
print("🔄 Word2Vec 모델 학습 중...")
w2v_model = Word2Vec(
    sentences=preprocessed_texts,  # 학습 데이터 (토큰화된 문장들)
    vector_size=100,              # 임베딩 벡터 차원 수
    window=5,                     # 컨텍스트 윈도우 크기
    min_count=1,                  # 최소 출현 빈도 (데이터가 적어서 1로 설정)
    workers=4,                    # 병렬 처리 워커 수
    sg=0,                        # 0=CBOW, 1=Skip-gram
    epochs=10                    # 학습 반복 횟수
)

print("✅ Word2Vec 모델 학습 완료!")

print(f"\n📊 Word2Vec 모델 정보:")
print(f"  어휘 크기: {len(w2v_model.wv)}개")
print(f"  벡터 차원: {w2v_model.wv.vector_size}차원")
print(f"  학습된 단어들: {list(w2v_model.wv.index_to_key)}")

print(f"\n🔍 단어 벡터 예시:")
# 학습된 모든 단어의 벡터 확인
for i, word in enumerate(w2v_model.wv.index_to_key[:5]):  # 처음 5개 단어만
    vector = w2v_model.wv[word]
    print(f"  '{word}' 벡터:")
    print(f"    전체 차원: {len(vector)}차원")
    print(f"    처음 10차원: {vector[:10]}")
    print(f"    벡터 크기(norm): {np.linalg.norm(vector):.4f}")
    print()

🔄 Word2Vec 모델 학습 중...
✅ Word2Vec 모델 학습 완료!

📊 Word2Vec 모델 정보:
  어휘 크기: 48개
  벡터 차원: 100차원
  학습된 단어들: ['nlp', 'text', 'models', 'language', 'learning', 'natural', 'many', 'process', 'revolutionized', 'tasks', 'deep', 'data', 'classification', 'amounts', 'large', 'analyze', 'algorithms', 'results', 'sentiment', 'machine', 'intelligence', 'artificial', 'field', 'fascinating', 'processing', 'analysis', 'understanding', 'preprocessing', 'essential', 'step', 'stateoftheart', 'achieve', 'gpt', 'bert', 'modern', 'architecture', 'dominant', 'become', 'transformers', 'words', 'relationships', 'semantic', 'capture', 'embeddings', 'word', 'pipeline', 'common', 'applications']

🔍 단어 벡터 예시:
  'nlp' 벡터:
    전체 차원: 100차원
    처음 10차원: [-0.00053441  0.0002342   0.00509681  0.00901455 -0.00930748 -0.00712235
  0.00645791  0.0089834  -0.00501605 -0.00377127]
    벡터 크기(norm): 0.0566

  'text' 벡터:
    전체 차원: 100차원
    처음 10차원: [-0.00861682  0.00366286  0.00518951  0.00575312  0.00745169 -0.00618391
  0.0011

In [19]:
def calculate_word_similarity(w2v_model):
    """단어 간 유사도 계산 및 설명"""
    print("📐 단어 간 유사도 계산:")
    print("유사도 = 코사인 유사도 (두 벡터 간 각도로 측정)")
    print("  1.0에 가까울수록 유사, 0에 가까울수록 관련 없음")
    print()
    
    # 학습된 단어들 중에서 유사도 계산
    vocab = list(w2v_model.wv.index_to_key)
    
    if len(vocab) >= 2:
        print("💡 단어 쌍별 유사도:")
        
        # 몇 가지 단어 쌍의 유사도 계산
        word_pairs = []
        
        # 'natural'과 다른 단어들 비교
        if 'natural' in vocab:
            for word in vocab:
                if word != 'natural':
                    word_pairs.append(('natural', word))
        
        # 'learning'과 다른 단어들 비교  
        if 'learning' in vocab:
            for word in vocab:
                if word != 'learning' and ('learning', word) not in word_pairs:
                    word_pairs.append(('learning', word))
        
        # 처음 몇 개만 보여주기
        for word1, word2 in word_pairs[:8]:
            try:
                similarity = w2v_model.wv.similarity(word1, word2)
                print(f"  '{word1}' ↔ '{word2}': {similarity:.4f}")
            except KeyError as e:
                print(f"  '{word1}' ↔ '{word2}': 계산 불가 ({e})")
    else:
        print("유사도 계산을 위한 충분한 어휘가 없습니다.")

calculate_word_similarity(w2v_model)

📐 단어 간 유사도 계산:
유사도 = 코사인 유사도 (두 벡터 간 각도로 측정)
  1.0에 가까울수록 유사, 0에 가까울수록 관련 없음

💡 단어 쌍별 유사도:
  'natural' ↔ 'nlp': -0.0597
  'natural' ↔ 'text': 0.0095
  'natural' ↔ 'models': 0.0649
  'natural' ↔ 'language': 0.1319
  'natural' ↔ 'learning': 0.1391
  'natural' ↔ 'many': 0.0192
  'natural' ↔ 'process': -0.0578
  'natural' ↔ 'revolutionized': 0.0610


In [20]:
def find_similar_words(w2v_model):
    """유사 단어 찾기"""
    print(f"\n🔎 유사 단어 찾기:")
    print("각 단어와 가장 유사한 단어들을 찾아보겠습니다.")
    print()
    
    vocab = list(w2v_model.wv.index_to_key)
    
    # 몇 개 단어에 대해 유사 단어 찾기
    target_words = ['learning', 'natural', 'language', 'processing', 'data']
    
    for target_word in target_words:
        if target_word in vocab:
            print(f"📝 '{target_word}'와 유사한 단어들:")
            try:
                # 가장 유사한 상위 3개 단어 찾기
                similar_words = w2v_model.wv.most_similar(target_word, topn=3)
                
                if similar_words:
                    for rank, (word, score) in enumerate(similar_words, 1):
                        print(f"  {rank}. '{word}': {score:.4f}")
                else:
                    print("  유사한 단어를 찾을 수 없습니다.")
                        
            except Exception as e:
                print(f"  오류: {e}")
            print()

find_similar_words(w2v_model)


🔎 유사 단어 찾기:
각 단어와 가장 유사한 단어들을 찾아보겠습니다.

📝 'learning'와 유사한 단어들:
  1. 'amounts': 0.2534
  2. 'understanding': 0.2008
  3. 'transformers': 0.1953

📝 'natural'와 유사한 단어들:
  1. 'data': 0.1671
  2. 'stateoftheart': 0.1632
  3. 'learning': 0.1391

📝 'language'와 유사한 단어들:
  1. 'large': 0.1783
  2. 'dominant': 0.1638
  3. 'pipeline': 0.1496

📝 'processing'와 유사한 단어들:
  1. 'semantic': 0.1924
  2. 'capture': 0.1566
  3. 'deep': 0.1020

📝 'data'와 유사한 단어들:
  1. 'stateoftheart': 0.1819
  2. 'models': 0.1729
  3. 'natural': 0.1671



In [21]:
def demonstrate_vector_arithmetic(w2v_model):
    """벡터 연산 데모"""
    print("🧮 벡터 연산 (단어 관계 학습):")
    print("Word2Vec의 놀라운 특성: 단어 간 관계를 벡터 연산으로 표현 가능!")
    print("예: king - man + woman = queen (왕 - 남자 + 여자 = 여왕)")
    print()
    
    vocab = list(w2v_model.wv.index_to_key)
    
    # 우리 데이터에서 가능한 벡터 연산 시도
    if len(vocab) >= 3:
        print("🔬 우리 데이터로 벡터 연산 실험:")
        
        # 몇 가지 조합 시도
        combinations = [
            ('natural', 'language', 'processing'),
            ('machine', 'learning', 'data'),
            ('deep', 'learning', 'model')
        ]
        
        for word1, word2, word3 in combinations:
            if all(word in vocab for word in [word1, word2, word3]):
                try:
                    # word1 - word2 + word3와 가장 유사한 단어 찾기
                    result = w2v_model.wv.most_similar(
                        positive=[word1, word3], 
                        negative=[word2], 
                        topn=1
                    )
                    if result:
                        result_word, score = result[0]
                        print(f"  '{word1}' - '{word2}' + '{word3}' ≈ '{result_word}' ({score:.4f})")
                except:
                    print(f"  '{word1}' - '{word2}' + '{word3}': 계산 실패")
    
    if not any(all(word in vocab for word in combo) for combo in combinations):
        print("벡터 연산을 위한 충분한 관련 단어들이 없습니다.")
        print("더 많은 데이터로 학습하면 더 흥미로운 결과를 볼 수 있습니다!")

demonstrate_vector_arithmetic(w2v_model)

🧮 벡터 연산 (단어 관계 학습):
Word2Vec의 놀라운 특성: 단어 간 관계를 벡터 연산으로 표현 가능!
예: king - man + woman = queen (왕 - 남자 + 여자 = 여왕)

🔬 우리 데이터로 벡터 연산 실험:
  'natural' - 'language' + 'processing' ≈ 'semantic' (0.2005)
  'machine' - 'learning' + 'data' ≈ 'process' (0.2366)


-----

### 6. GloVe (사전 훈련된 모델 사용)

GloVe 개념:
- Stanford에서 개발한 단어 임베딩 기법
- Word2Vec과 달리 전역적(Global) 통계 정보를 활용
- 단어-단어 동시출현 행렬(Co-occurrence Matrix)을 기반으로 학습
- Word2Vec의 예측 기반 방법과 전통적인 통계 방법을 결합

GloVe의 핵심 아이디어:
- "단어의 의미는 전체 말뭉치에서의 동시출현 패턴으로 결정된다"
- 예: '얼음'과 '증기' 모두 '물'과 자주 나타나지만, '고체'는 '얼음'과 더 관련있음
- 이러한 비율 정보를 벡터로 학습

Word2Vec vs GloVe:
- Word2Vec: 지역적 문맥 윈도우 내에서 예측 학습
- GloVe: 전체 말뭉치의 통계적 정보를 직접 활용

In [22]:
print("🔄 GloVe 사전 훈련된 모델 로드 시작...")
print("⏱️  첫 실행 시 인터넷에서 다운로드하므로 시간이 걸릴 수 있습니다.")
print("📦 모델 크기: 약 50MB (glove-wiki-gigaword-50)")

# 작은 GloVe 모델 로드 (50차원)
print("\n📥 Gensim API를 통해 GloVe 모델 다운로드 중...")
glove_model = api.load("glove-wiki-gigaword-50")

print("✅ GloVe 모델 로드 성공!")

🔄 GloVe 사전 훈련된 모델 로드 시작...
⏱️  첫 실행 시 인터넷에서 다운로드하므로 시간이 걸릴 수 있습니다.
📦 모델 크기: 약 50MB (glove-wiki-gigaword-50)

📥 Gensim API를 통해 GloVe 모델 다운로드 중...
✅ GloVe 모델 로드 성공!


In [23]:
# 모델 기본 정보 출력
print(f"\n📊 GloVe 모델 정보:")
print(f"  어휘 크기: {len(glove_model):,}개 단어")
print(f"  벡터 차원: 50차원")
print(f"  학습 데이터: Wikipedia + Gigaword (60억 토큰)")

# 어휘 예시 확인
print(f"\n📖 어휘 예시 (처음 20개):")
vocab_sample = list(glove_model.index_to_key[:20])
for i, word in enumerate(vocab_sample):
    print(f"  {i+1:2d}. '{word}'")


📊 GloVe 모델 정보:
  어휘 크기: 400,000개 단어
  벡터 차원: 50차원
  학습 데이터: Wikipedia + Gigaword (60억 토큰)

📖 어휘 예시 (처음 20개):
   1. 'the'
   2. ','
   3. '.'
   4. 'of'
   5. 'to'
   6. 'and'
   7. 'in'
   8. 'a'
   9. '"'
  10. ''s'
  11. 'for'
  12. '-'
  13. 'that'
  14. 'on'
  15. 'is'
  16. 'was'
  17. 'said'
  18. 'with'
  19. 'he'
  20. 'as'


In [24]:
def demonstrate_word_vectors(preprocessed_texts):
    """단어 벡터 예시"""
    print(f"\n🔍 단어 벡터 예시:")

    for word in preprocessed_texts:
        if word in glove_model:
            vector = glove_model[word]
            print(f"  📝 '{word}' 벡터:")
            print(f"    전체 차원: {len(vector)}차원")
            print(f"    처음 10차원: {vector[:10]}")
            print(f"    벡터 크기: {np.linalg.norm(vector):.4f}")
            print(f"    최대값: {np.max(vector):.4f}, 최소값: {np.min(vector):.4f}")
            print()
        else:
            print(f"  ❌ '{word}'는 어휘에 없습니다.")

demonstrate_word_vectors(preprocessed_texts[0])


🔍 단어 벡터 예시:
  📝 'natural' 벡터:
    전체 차원: 50차원
    처음 10차원: [ 0.44265  0.84765 -0.4598   0.67993  0.13841  0.39456 -0.17343 -0.64055
  0.86439  0.81624]
    벡터 크기: 5.0903
    최대값: 3.3077, 최소값: -0.9165

  📝 'language' 벡터:
    전체 차원: 50차원
    처음 10차원: [-0.5799    -0.1101    -1.1557    -0.0029906 -0.20613    0.45289
 -0.16671   -1.0382    -0.99241    0.39884  ]
    벡터 크기: 6.0993
    최대값: 3.7163, 최소값: -1.3878

  📝 'processing' 벡터:
    전체 차원: 50차원
    처음 10차원: [ 1.6092e-01 -9.0221e-01  1.5797e-01  1.1776e+00 -6.2201e-04 -1.9004e-02
 -1.5081e-01 -5.8863e-01  1.5128e+00  4.2868e-01]
    벡터 크기: 5.3926
    최대값: 3.2094, 최소값: -0.9775

  📝 'fascinating' 벡터:
    전체 차원: 50차원
    처음 10차원: [ 0.90512   0.63951  -0.94111   0.33322   1.0375   -0.060236  0.043731
 -0.26376   0.074989  0.8521  ]
    벡터 크기: 4.6380
    최대값: 1.6140, 최소값: -1.6849

  📝 'field' 벡터:
    전체 차원: 50차원
    처음 10차원: [-0.49284  0.3731   0.15565  0.70044  0.77405 -0.48151 -1.2244  -0.02163
  0.63856 -0.49535]
    벡터 크기: 5.1761
    최대값: 

In [25]:
# 의미적으로 관련된 단어쌍들
word_pairs = [
    ('artificial', 'intelligence'),
    ('computer', 'technology'),
    ('language', 'communication'),
    ('learning', 'education'),
    ('deep', 'shallow'),
    ('king', 'queen'),
    ('man', 'woman'),
    ('car', 'vehicle')
]

def calculate_similarities(word_pairs):
    """단어 간 유사도 계산"""
    print("📐 단어 간 유사도 계산:")
    print("유사도가 높을수록 의미적으로 관련이 깊은 단어들입니다.")
    print()

    print("💡 단어 쌍별 유사도:")
    for word1, word2 in word_pairs:
        if word1 in glove_model and word2 in glove_model:
            similarity = glove_model.similarity(word1, word2)
            print(f"  '{word1}' ↔ '{word2}': {similarity:.4f}")
            
            # 유사도 해석
            if similarity > 0.7:
                print(f"    → 매우 높은 유사도 (강한 관련성)")
            elif similarity > 0.5:
                print(f"    → 높은 유사도 (관련성 있음)")
            elif similarity > 0.3:
                print(f"    → 중간 유사도 (약간 관련)")
            else:
                print(f"    → 낮은 유사도 (관련성 적음)")
        else:
            missing_words = [w for w in [word1, word2] if w not in glove_model]
            print(f"  '{word1}' ↔ '{word2}': 계산 불가 (어휘에 없음: {missing_words})")
        print()

calculate_similarities(word_pairs)

📐 단어 간 유사도 계산:
유사도가 높을수록 의미적으로 관련이 깊은 단어들입니다.

💡 단어 쌍별 유사도:
  'artificial' ↔ 'intelligence': 0.1664
    → 낮은 유사도 (관련성 적음)

  'computer' ↔ 'technology': 0.8526
    → 매우 높은 유사도 (강한 관련성)

  'language' ↔ 'communication': 0.5878
    → 높은 유사도 (관련성 있음)

  'learning' ↔ 'education': 0.7518
    → 매우 높은 유사도 (강한 관련성)

  'deep' ↔ 'shallow': 0.7442
    → 매우 높은 유사도 (강한 관련성)

  'king' ↔ 'queen': 0.7839
    → 매우 높은 유사도 (강한 관련성)

  'man' ↔ 'woman': 0.8860
    → 매우 높은 유사도 (강한 관련성)

  'car' ↔ 'vehicle': 0.8834
    → 매우 높은 유사도 (강한 관련성)



벡터 간 유사도 계산이라는 기본 원리를 다양한 실전 문제에 응용
- 검색 시스템 - 사용자가 "노트북" 검색해도 "laptop" 문서 찾아줌
- 추천 시스템 - 넷플릭스, 스포티파이 같은 개인화 추천
- 감정 분석 - 고객 리뷰, SNS 모니터링 자동화
- 챗봇 - 다양한 표현을 이해하는 스마트 고객 서비스
- 사기 탐지 - 피싱 메일, 허위 리뷰 자동 감지

In [26]:
target_words = ['computer', 'artificial', 'language', 'learning', 'science']

def find_similar_words(target_words):
    """유사 단어 찾기"""
    print("🔎 유사 단어 찾기:")
    print("각 단어와 의미적으로 가장 가까운 단어들을 찾아보겠습니다.")
    print()
    
    for word in target_words:
        if word in glove_model:
            print(f"📝 '{word}'와 가장 유사한 단어들:")
            try:
                # 상위 5개 유사 단어 찾기
                similar_words = glove_model.most_similar(word, topn=5)
                
                for rank, (similar_word, score) in enumerate(similar_words, 1):
                    print(f"  {rank}. '{similar_word}': {score:.4f}")
                    
            except Exception as e:
                print(f"  오류 발생: {e}")
            print()
        else:
            print(f"❌ '{word}'는 어휘에 없습니다.")
            print()

find_similar_words(target_words)


🔎 유사 단어 찾기:
각 단어와 의미적으로 가장 가까운 단어들을 찾아보겠습니다.

📝 'computer'와 가장 유사한 단어들:
  1. 'computers': 0.9165
  2. 'software': 0.8815
  3. 'technology': 0.8526
  4. 'electronic': 0.8126
  5. 'internet': 0.8060

📝 'artificial'와 가장 유사한 단어들:
  1. 'natural': 0.7425
  2. 'tissue': 0.7081
  3. 'synthetic': 0.7078
  4. 'developed': 0.6942
  5. 'therapeutic': 0.6857

📝 'language'와 가장 유사한 단어들:
  1. 'languages': 0.8815
  2. 'word': 0.8100
  3. 'spoken': 0.8075
  4. 'vocabulary': 0.7903
  5. 'translation': 0.7879

📝 'learning'와 가장 유사한 단어들:
  1. 'teaching': 0.8757
  2. 'skills': 0.8351
  3. 'experience': 0.8201
  4. 'practical': 0.8190
  5. 'knowledge': 0.8090

📝 'science'와 가장 유사한 단어들:
  1. 'sciences': 0.8548
  2. 'research': 0.8437
  3. 'institute': 0.8386
  4. 'studies': 0.8369
  5. 'physics': 0.8314



In [27]:
# 유명한 단어 유추 예제들
analogies = [
    ('king', 'man', 'queen'),      # king - man + woman = queen 의도
    ('paris', 'france', 'london'), # paris - france + england = london 의도  
    ('good', 'better', 'bad'),     # good - better + worse = bad 의도
    ('walking', 'walked', 'running'), # walking - walked + ran = running 의도
]

def demonstrate_word_analogies(analogies):
    """단어 유추 (Word Analogies) 데모"""
    print("🧮 단어 유추 (Word Analogies):")
    print("GloVe의 놀라운 능력: 단어 간 관계를 벡터 연산으로 표현!")
    print("형태: A : B = C : ?  (A가 B에 대응되는 것처럼, C는 무엇에 대응될까?)")
    print()
    
    print("🔬 단어 유추 실험:")
    for word1, word2, word3 in analogies:
        # 모든 단어가 어휘에 있는지 확인
        if all(word in glove_model for word in [word1, word2, word3]):
            try:
                # word1 - word2 + word3 와 가장 유사한 단어 찾기
                # positive=[word1, word3]: 더하는 벡터들
                # negative=[word2]: 빼는 벡터
                result = glove_model.most_similar(
                    positive=[word1, word3], 
                    negative=[word2], 
                    topn=3
                )
                
                print(f"  🎯 '{word1}' - '{word2}' + '{word3}' = ?")
                print(f"    결과 후보:")
                for rank, (result_word, score) in enumerate(result, 1):
                    print(f"      {rank}. '{result_word}': {score:.4f}")
                
                # 첫 번째 결과가 의도한 답인지 확인
                if result:
                    best_answer = result[0][0]
                    expected_answers = {
                        ('king', 'man', 'queen'): 'woman',
                        ('paris', 'france', 'london'): 'england', 
                        ('good', 'better', 'bad'): 'worse',
                        ('walking', 'walked', 'running'): 'ran'
                    }
                    expected = expected_answers.get((word1, word2, word3))
                    if expected and expected in [r[0] for r in result[:3]]:
                        print(f"    ✅ 기대한 답 '{expected}'이 상위 3개 안에 있습니다!")
                    else:
                        print(f"    💭 기대한 답은 '{expected}'이었지만 다른 결과가 나왔습니다.")
                
            except Exception as e:
                print(f"  ❌ '{word1}' - '{word2}' + '{word3}': 계산 실패 ({e})")
            print()
        else:
            missing = [w for w in [word1, word2, word3] if w not in glove_model]
            print(f"  ❌ '{word1}' - '{word2}' + '{word3}': 어휘에 없는 단어 {missing}")
            print()

demonstrate_word_analogies(analogies)

🧮 단어 유추 (Word Analogies):
GloVe의 놀라운 능력: 단어 간 관계를 벡터 연산으로 표현!
형태: A : B = C : ?  (A가 B에 대응되는 것처럼, C는 무엇에 대응될까?)

🔬 단어 유추 실험:
  🎯 'king' - 'man' + 'queen' = ?
    결과 후보:
      1. 'coronation': 0.7995
      2. 'hrh': 0.7570
      3. 'throne': 0.7358
    💭 기대한 답은 'woman'이었지만 다른 결과가 나왔습니다.

  🎯 'paris' - 'france' + 'london' = ?
    결과 후보:
      1. 'opened': 0.7355
      2. 'at': 0.7293
      3. 'hotel': 0.7116
    💭 기대한 답은 'england'이었지만 다른 결과가 나왔습니다.

  🎯 'good' - 'better' + 'bad' = ?
    결과 후보:
      1. 'little': 0.8396
      2. 'luck': 0.8365
      3. 'thing': 0.8204
    💭 기대한 답은 'worse'이었지만 다른 결과가 나왔습니다.

  🎯 'walking' - 'walked' + 'running' = ?
    결과 후보:
      1. 'turning': 0.7794
      2. 'track': 0.7608
      3. 'course': 0.7578
    💭 기대한 답은 'ran'이었지만 다른 결과가 나왔습니다.



#### 완벽한 답이 안나오는 이유?
1. 모델 크기의 한계
- 우리가 사용한 glove-wiki-gigaword-50은 50차원짜리 작은 모델
- 유명한 "king - man + woman = queen" 예제는 보통 300차원 모델에서 잘 작동
- 차원이 적으면 미묘한 의미 관계를 제대로 포착하기 어려움

2. 벡터 연산의 한계
- 벡터 공간에서는 여러 의미 관계가 복합적으로 얽혀있음
- "king - man + woman"이 정확히 "queen"만을 가리키지 않음
- 왕실, 권력, 성별 등 여러 개념이 동시에 활성화됨

'king' - 'man' + 'queen' = ?<br>
결과: 'coronation', 'hrh', 'throne'<br>
→ 실제로는 '왕권', '왕실', '왕좌' 관련 단어들!<br>
→ 의미적으로는 맞는 답! 단지 우리가 원한 답이 아닐 뿐

In [28]:
def compare_with_word2vec(w2v_model, glove_model):
    """Word2Vec과 GloVe 비교"""
    print("⚖️ Word2Vec vs GloVe 성능 비교:")
    print("="*50)
    
    # 공통 단어로 비교 (둘 다 있는 단어 찾기)
    w2v_vocab = set(w2v_model.wv.index_to_key)
    glove_vocab = set(glove_model.index_to_key)
    common_words = w2v_vocab.intersection(glove_vocab)
    
    if len(common_words) >= 2:
        print(f"📊 공통 어휘: {len(common_words)}개 단어")
        
        # 몇 개 단어 쌍의 유사도 비교
        test_pairs = []
        common_list = list(common_words)
        for i in range(min(3, len(common_list))):
            for j in range(i+1, min(i+3, len(common_list))):
                test_pairs.append((common_list[i], common_list[j]))
        
        print(f"\n📐 유사도 비교 (상위 {len(test_pairs)}개 쌍):")
        print(f"{'단어쌍':<20} {'Word2Vec':<12} {'GloVe':<12} {'차이':<10}")
        print("-" * 60)
        
        for word1, word2 in test_pairs:
            try:
                w2v_sim = w2v_model.wv.similarity(word1, word2)
                glove_sim = glove_model.similarity(word1, word2)
                diff = abs(w2v_sim - glove_sim)
                
                pair_name = f"{word1}-{word2}"
                print(f"{pair_name:<20} {w2v_sim:<12.4f} {glove_sim:<12.4f} {diff:<10.4f}")
            except:
                print(f"{word1}-{word2:<15} 계산 오류")
    else:
        print("Word2Vec과 GloVe 간 공통 어휘가 부족합니다.")
        
compare_with_word2vec(w2v_model, glove_model)

⚖️ Word2Vec vs GloVe 성능 비교:
📊 공통 어휘: 47개 단어

📐 유사도 비교 (상위 6개 쌍):
단어쌍                  Word2Vec     GloVe        차이        
------------------------------------------------------------
models-learning      0.1707       0.4946       0.3239    
models-semantic      0.2123       0.4127       0.2003    
learning-semantic    -0.1680      0.4759       0.6439    
learning-many        0.0347       0.6012       0.5665    
semantic-many        0.0794       0.1504       0.0711    
semantic-fascinating -0.0221      0.3300       0.3521    


### 7. FastText (Fast Text Representation)
FastText 개념:
- Facebook에서 개발한 단어 임베딩 기법
- Word2Vec의 진화된 버전
- 핵심 차이점: 하위 단어(subword) 정보를 활용
- 단어를 문자 n-gram으로 분해해서 학습

FastText의 혁신적 아이디어:
- Word2Vec: "apple" → 하나의 벡터
- FastText: "apple" → "<ap", "app", "ppl", "ple", "le>" + "apple" 전체
- 장점: 훈련에 없던 단어도 하위 단어 조합으로 벡터 생성 가능!

하위 단어(Subword) 예시:
- "running" → "<ru", "run", "unn", "nni", "nin", "ing", "ng>"
- "unknown" (새 단어) → "<un", "unk", "nkn", "kno", "now", "own", "wn>"
- 비슷한 철자 패턴 → 비슷한 의미 추론

<br>

FastText vs Word2Vec 비교:
<br>

Word2Vec의 한계:
OOV (Out-of-Vocabulary) 문제:
- 훈련 데이터에 없는 단어 → 벡터 생성 불가
- 예: 'smartphone' 학습했지만 'smartphones' 없으면 처리 못함
- 오타, 신조어, 전문용어 처리 어려움

<br>

FastText의 해결책:
하위 단어 기반 학습:
- 단어를 작은 조각들로 분해
- 조각들의 조합으로 새로운 단어 벡터 생성
- 예: 'run' + 'ning' 패턴으로 'running' 이해

In [29]:
def show_subword_process(example_words):
    """하위 단어 분해 과정 시연"""
    print(f"\n🔍 하위 단어 분해 과정 시연:")
    print("-" * 40)
    
    # 예시 단어들  
    print("📝 단어별 하위 단어 분해 (n-gram=3~6):")
    
    for word in example_words:
        print(f"\n  '{word}' 분해:")
        
        # 시작/끝 마커 추가
        word_with_markers = f"<{word}>"
        print(f"    마커 추가: {word_with_markers}")
        
        # 3-gram부터 6-gram까지 생성
        subwords = []
        
        for n in range(3, min(7, len(word_with_markers) + 1)):
            for i in range(len(word_with_markers) - n + 1):
                subword = word_with_markers[i:i+n]
                subwords.append(subword)
        
        # 전체 단어도 포함
        subwords.append(word)
        
        print(f"    하위 단어들: {subwords[:10]}...")  # 처음 10개만 표시
        print(f"    총 {len(subwords)}개 조각")

show_subword_process(preprocessed_texts[0])


🔍 하위 단어 분해 과정 시연:
----------------------------------------
📝 단어별 하위 단어 분해 (n-gram=3~6):

  'natural' 분해:
    마커 추가: <natural>
    하위 단어들: ['<na', 'nat', 'atu', 'tur', 'ura', 'ral', 'al>', '<nat', 'natu', 'atur']...
    총 23개 조각

  'language' 분해:
    마커 추가: <language>
    하위 단어들: ['<la', 'lan', 'ang', 'ngu', 'gua', 'uag', 'age', 'ge>', '<lan', 'lang']...
    총 27개 조각

  'processing' 분해:
    마커 추가: <processing>
    하위 단어들: ['<pr', 'pro', 'roc', 'oce', 'ces', 'ess', 'ssi', 'sin', 'ing', 'ng>']...
    총 35개 조각

  'fascinating' 분해:
    마커 추가: <fascinating>
    하위 단어들: ['<fa', 'fas', 'asc', 'sci', 'cin', 'ina', 'nat', 'ati', 'tin', 'ing']...
    총 39개 조각

  'field' 분해:
    마커 추가: <field>
    하위 단어들: ['<fi', 'fie', 'iel', 'eld', 'ld>', '<fie', 'fiel', 'ield', 'eld>', '<fiel']...
    총 15개 조각

  'artificial' 분해:
    마커 추가: <artificial>
    하위 단어들: ['<ar', 'art', 'rti', 'tif', 'ifi', 'fic', 'ici', 'cia', 'ial', 'al>']...
    총 35개 조각

  'intelligence' 분해:
    마커 추가: <intelligence>
    하위 단어들: 

In [30]:
print(f"\n🚀 FastText 모델 학습 시작!")
print("="*50)
print("🔄 FastText 모델 학습 중...")
print("⏱️  Word2Vec보다 약간 더 오래 걸릴 수 있습니다 (하위 단어 처리 때문)")

fasttext_model = FastText(
    sentences=preprocessed_texts,  # 학습 데이터 (토큰화된 문장들)
    vector_size=100,              # 임베딩 벡터 차원 수
    window=5,                     # 컨텍스트 윈도우 크기
    min_count=1,                  # 최소 출현 빈도 (데이터가 적어서 1로 설정)
    workers=4,                    # 병렬 처리 워커 수
    sg=0,                        # 0=CBOW, 1=Skip-gram
    epochs=10                    # 학습 반복 횟수
)

print("✅ FastText 모델 학습 완료!")

print(f"\n📊 FastText 모델 정보:")
print(f"  어휘 크기: {len(fasttext_model.wv)}개")
print(f"  벡터 차원: {fasttext_model.wv.vector_size}차원")
print(f"  학습된 단어들: {list(fasttext_model.wv.index_to_key)}")


🚀 FastText 모델 학습 시작!
🔄 FastText 모델 학습 중...
⏱️  Word2Vec보다 약간 더 오래 걸릴 수 있습니다 (하위 단어 처리 때문)
✅ FastText 모델 학습 완료!

📊 FastText 모델 정보:
  어휘 크기: 48개
  벡터 차원: 100차원
  학습된 단어들: ['nlp', 'text', 'models', 'language', 'learning', 'natural', 'many', 'process', 'revolutionized', 'tasks', 'deep', 'data', 'classification', 'amounts', 'large', 'analyze', 'algorithms', 'results', 'sentiment', 'machine', 'intelligence', 'artificial', 'field', 'fascinating', 'processing', 'analysis', 'understanding', 'preprocessing', 'essential', 'step', 'stateoftheart', 'achieve', 'gpt', 'bert', 'modern', 'architecture', 'dominant', 'become', 'transformers', 'words', 'relationships', 'semantic', 'capture', 'embeddings', 'word', 'pipeline', 'common', 'applications']


In [31]:
def demonstrate_basic_functionality(fasttext_model):
    """기본 기능 시연"""
    print(f"\n🔍 기본 기능 확인:")
    print("-" * 40)
    
    # 학습된 단어의 벡터 확인
    vocab = list(fasttext_model.wv.index_to_key)
    
    if vocab:
        test_word = vocab[0]
        vector = fasttext_model.wv[test_word]
        print(f"📝 '{test_word}' 벡터:")
        print(f"  전체 차원: {len(vector)}차원")
        print(f"  처음 10차원: {vector[:10]}")
        print(f"  벡터 크기: {np.linalg.norm(vector):.4f}")
    
    # 단어 유사도 계산
    if len(vocab) >= 2:
        word1, word2 = vocab[0], vocab[1]
        try:
            similarity = fasttext_model.wv.similarity(word1, word2)
            print(f"\n📐 단어 유사도:")
            print(f"  '{word1}' ↔ '{word2}': {similarity:.4f}")
        except:
            print(f"  유사도 계산 실패")

demonstrate_basic_functionality(fasttext_model)


🔍 기본 기능 확인:
----------------------------------------
📝 'nlp' 벡터:
  전체 차원: 100차원
  처음 10차원: [ 0.00229694 -0.00158578  0.00267106  0.00239871 -0.0020173   0.00127126
  0.00130897  0.00026072 -0.00232229 -0.00022632]
  벡터 크기: 0.0253

📐 단어 유사도:
  'nlp' ↔ 'text': -0.0781


In [32]:
# OOV 단어들 테스트
oov_test_words = [
    "artificialintelligence",  # 'artificial' + 'intelligence' 합성
    "machinelearning",         # 'machine' + 'learning' 합성  
    "deeplearning",           # 'deep' + 'learning' 합성
    "naturallanguage",        # 'natural' + 'language' 합성
    "smartphone",             # 일반적인 신조어
    "neuralnetwork"           # 'neural' + 'network' 합성
]

def demonstrate_oov_handling(oov_test_words):
    """OOV (미등록 단어) 처리 시연 - FastText의 핵심 기능!"""
    print(f"\n🎯 FastText의 핵심 기능: OOV 단어 처리")
    print("="*50)
    
    print("🔍 OOV (Out-of-Vocabulary) 문제란?")
    print("  - 훈련 데이터에 없던 새로운 단어를 만났을 때")
    print("  - Word2Vec: '단어를 모르겠다' → 오류")
    print("  - FastText: '하위 단어로 추론해볼게' → 벡터 생성!")
    
    # 훈련 데이터에 있는 단어 확인
    vocab = set(fasttext_model.wv.index_to_key)
    print(f"\n📚 훈련된 어휘: {vocab}")
    print(f"\n🧪 OOV 단어 테스트:")
    print("훈련 데이터에 없는 단어들도 벡터 생성이 가능한지 확인해보겠습니다.")
    
    for oov_word in oov_test_words:
        try:
            # FastText는 훈련에 없던 단어도 벡터 생성 가능!
            oov_vector = fasttext_model.wv[oov_word]
            
            print(f"\n✅ '{oov_word}' (OOV 단어):")
            print(f"  벡터 생성 성공! 차원: {len(oov_vector)}")
            print(f"  처음 5차원: {oov_vector[:5]}")
            print(f"  벡터 크기: {np.linalg.norm(oov_vector):.4f}")
            
            # 훈련된 단어들과 유사도 계산
            if vocab:
                similarities = []
                for known_word in list(vocab)[:3]:  # 처음 3개 단어와 비교
                    try:
                        sim = fasttext_model.wv.similarity(oov_word, known_word)
                        similarities.append((known_word, sim))
                    except:
                        continue
                
                if similarities:
                    similarities.sort(key=lambda x: x[1], reverse=True)
                    print(f"  가장 유사한 훈련 단어: '{similarities[0][0]}' ({similarities[0][1]:.4f})")
                    
        except Exception as e:
            print(f"❌ '{oov_word}': 벡터 생성 실패 - {e}")

demonstrate_oov_handling(oov_test_words)


🎯 FastText의 핵심 기능: OOV 단어 처리
🔍 OOV (Out-of-Vocabulary) 문제란?
  - 훈련 데이터에 없던 새로운 단어를 만났을 때
  - Word2Vec: '단어를 모르겠다' → 오류
  - FastText: '하위 단어로 추론해볼게' → 벡터 생성!

📚 훈련된 어휘: {'models', 'learning', 'semantic', 'many', 'fascinating', 'architecture', 'natural', 'language', 'classification', 'modern', 'artificial', 'essential', 'results', 'intelligence', 'relationships', 'understanding', 'become', 'step', 'embeddings', 'word', 'field', 'words', 'dominant', 'preprocessing', 'bert', 'process', 'processing', 'analyze', 'achieve', 'applications', 'transformers', 'machine', 'gpt', 'common', 'sentiment', 'deep', 'pipeline', 'stateoftheart', 'algorithms', 'amounts', 'capture', 'text', 'revolutionized', 'nlp', 'analysis', 'tasks', 'data', 'large'}

🧪 OOV 단어 테스트:
훈련 데이터에 없는 단어들도 벡터 생성이 가능한지 확인해보겠습니다.

✅ 'artificialintelligence' (OOV 단어):
  벡터 생성 성공! 차원: 100
  처음 5차원: [ 0.0001186   0.00144737  0.00098309 -0.00015574  0.00026451]
  벡터 크기: 0.0056
  가장 유사한 훈련 단어: 'semantic' (-0.1166)

✅ 'machinelearning' (O

In [33]:
test_oov_word = "artificialintelligence"

def compare_with_word2vec(test_oov_word):
    """Word2Vec과 FastText OOV 처리 비교"""
    print(f"\n⚖️ Word2Vec vs FastText: OOV 처리 비교")
    print("="*50)
    
    print(f"🧪 테스트 단어: '{test_oov_word}'")
    print("(훈련 데이터에 없는 합성어)")
    
    # Word2Vec 테스트
    if 'w2v_model' in globals():
        print(f"\n📊 Word2Vec 결과:")
        try:
            w2v_vector = w2v_model.wv[test_oov_word]
            print(f"  ✅ 벡터 생성 성공 (처음 5차원): {w2v_vector[:5]}")
        except KeyError:
            print(f"  ❌ KeyError: 단어를 찾을 수 없습니다.")
            print(f"  → Word2Vec은 훈련에 없던 단어를 처리할 수 없음")
    else:
        print(f"\n📊 Word2Vec 모델이 없어 비교할 수 없습니다.")
    
    # FastText 테스트
    print(f"\n📊 FastText 결과:")
    try:
        ft_vector = fasttext_model.wv[test_oov_word]
        print(f"  ✅ 벡터 생성 성공 (처음 5차원): {ft_vector[:5]}")
        print(f"  → FastText는 하위 단어 조합으로 벡터 생성!")
        
        # 어떤 하위 단어들이 사용되었는지 추론
        print(f"  🔍 추론 과정:")
        print(f"    'artificial' 부분의 하위 단어들: <ar, art, rti, tif, ...")
        print(f"    'intelligence' 부분의 하위 단어들: int, nte, tel, ell, ...")
        print(f"    → 이런 조각들의 조합으로 전체 단어 의미 추론")
        
    except Exception as e:
        print(f"  ❌ 오류: {e}")

compare_with_word2vec(test_oov_word)


⚖️ Word2Vec vs FastText: OOV 처리 비교
🧪 테스트 단어: 'artificialintelligence'
(훈련 데이터에 없는 합성어)

📊 Word2Vec 결과:
  ❌ KeyError: 단어를 찾을 수 없습니다.
  → Word2Vec은 훈련에 없던 단어를 처리할 수 없음

📊 FastText 결과:
  ✅ 벡터 생성 성공 (처음 5차원): [ 0.0001186   0.00144737  0.00098309 -0.00015574  0.00026451]
  → FastText는 하위 단어 조합으로 벡터 생성!
  🔍 추론 과정:
    'artificial' 부분의 하위 단어들: <ar, art, rti, tif, ...
    'intelligence' 부분의 하위 단어들: int, nte, tel, ell, ...
    → 이런 조각들의 조합으로 전체 단어 의미 추론


In [34]:
print(f"\n💡 FastText 실습 요약:")
print("="*50)

print(f"📈 학습 결과:")
print(f"  어휘 크기: {len(fasttext_model.wv)}개 단어")
print(f"  벡터 차원: {fasttext_model.wv.vector_size}차원")
print(f"  핵심 특징: 하위 단어 기반 OOV 처리")


💡 FastText 실습 요약:
📈 학습 결과:
  어휘 크기: 48개 단어
  벡터 차원: 100차원
  핵심 특징: 하위 단어 기반 OOV 처리


----

### 최근 텍스트 전처리 방법: Transformers (BERT 등)

### 8. 현대적인 방법: Transformers (BERT)

Transformers & BERT 소개:
- 2017년 "Attention Is All You Need" 논문으로 시작된 혁명
- BERT (Bidirectional Encoder Representations from Transformers)
- 기존 방법들과의 핵심 차이점: 문맥을 양방향으로 이해

기존 방법들과의 비교:
- Word2Vec/GloVe/FastText: 단어별 고정 벡터
→ "bank" 단어는 항상 같은 벡터 (은행? 강둑?)
  
- BERT: 문맥에 따라 동적 벡터  
→ "I went to the bank" vs "river bank"에서 다른 벡터!

BERT의 혁신적 특징:
- 1. 양방향성: 앞뒤 문맥을 모두 고려
- 2. 문맥 의존성: 같은 단어도 문맥에 따라 다른 의미
- 3. 전이 학습: 대용량 데이터로 미리 훈련 → 특정 태스크에 적용
- 4. 문장 레벨 이해: 단어가 아닌 전체 문장의 의미 파악


#### 왜 Transformers가 NLP 역사를 바꿨을까?
성능의 비약적 향상:
- 기존 모델: 특정 태스크별로 따로 설계
- BERT: 하나의 모델로 여러 태스크 해결
- 결과: 거의 모든 NLP 벤치마크에서 기록 경신

인간과 유사한 언어 이해:
- 단어 하나하나가 아니라 문장 전체 맥락 파악
- '이 말이 이 상황에서 무슨 뜻일까?' 추론 가능
- 암묵적 의미, 반어법, 문맥 의존적 의미도 어느 정도 이해

전이 학습의 위력:
- 1단계: 수십억 개 문장으로 언어 자체를 학습
- 2단계: 특정 태스크 데이터로 미세 조정
- 결과: 적은 데이터로도 높은 성능 달성

#### BERT 구조 개요
핵심 구성 요소:
- 1. 토크나이저 (Tokenizer):
     - 문장을 BERT가 이해할 수 있는 토큰으로 변환
     - 예: 'Hello world' → ['[CLS]', 'Hello', 'world', '[SEP]']
- 2. 임베딩 레이어:
     - 토큰을 벡터로 변환 (단어 + 위치 + 문장 정보)
- 3. 트랜스포머 블록들:
     - 여러 층의 attention 메커니즘
     - 각 단어가 다른 모든 단어들과 '대화'하며 의미 파악
- 4. 출력:
     - [CLS]: 전체 문장의 의미 (분류 태스크용)
     - 각 토큰별: 문맥을 고려한 단어별 벡터


In [35]:
from transformers import AutoTokenizer, AutoModel
import torch

모델 선택: DistilBERT

BERT 모델 크기 비교:
- BERT-base: 110M 파라미터, 768차원, 12층
- BERT-large: 340M 파라미터, 1024차원, 24층
- DistilBERT: 66M 파라미터, 768차원, 6층 ← 우리가 사용

- DistilBERT 선택 이유:
  - 크기: BERT-base의 60% 크기
  - 속도: 약 60% 빠름
  - 성능: BERT-base의 97% 성능 유지
  - 메모리: 일반 PC에서도 실행 가능
<br>
  → 학습용으로 최적!

In [36]:
print("\n🔄 BERT 모델 로드 중...")
print("⏱️  처음 실행 시 인터넷에서 다운로드 (시간 소요)")

# 모델 이름 지정
model_name = "distilbert-base-uncased"
print(f"📥 사용 모델: {model_name}")
print("  - distilbert: 경량화된 BERT")
print("  - base: 기본 크기 (large 대비)")
print("  - uncased: 대소문자 구분 안함")

# 토크나이저 로드
print("\n1️⃣ 토크나이저 로드 중...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("✅ 토크나이저 로드 완료!")

# 모델 로드
print("\n2️⃣ BERT 모델 로드 중...")
model = AutoModel.from_pretrained(model_name)
print("✅ BERT 모델 로드 완료!")

print(f"\n📊 모델 정보:")
print(f"  모델 이름: {model_name}")
print(f"  파라미터 수: 약 66M개")
print(f"  출력 차원: 768차원")


🔄 BERT 모델 로드 중...
⏱️  처음 실행 시 인터넷에서 다운로드 (시간 소요)
📥 사용 모델: distilbert-base-uncased
  - distilbert: 경량화된 BERT
  - base: 기본 크기 (large 대비)
  - uncased: 대소문자 구분 안함

1️⃣ 토크나이저 로드 중...
✅ 토크나이저 로드 완료!

2️⃣ BERT 모델 로드 중...
✅ BERT 모델 로드 완료!

📊 모델 정보:
  모델 이름: distilbert-base-uncased
  파라미터 수: 약 66M개
  출력 차원: 768차원


In [37]:
example_text = "Natural language processing is amazing!"

def demonstrate_tokenization(example_text):
    """토큰화 과정 시연"""
    print("\n🔍 토큰화 과정 시연")
    print("-" * 30)
    
    print(f"원본 문장: '{example_text}'")
    
    # 토큰화 수행
    tokens = tokenizer.tokenize(example_text)
    print(f"토큰화 결과: {tokens}")
    
    # 토큰 ID로 변환
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    print(f"토큰 ID: {token_ids}")
    
    # 특수 토큰 추가된 버전
    encoded = tokenizer(example_text, return_tensors="pt")
    print(f"특수 토큰 포함: {encoded['input_ids']}")
    
    # 디코딩 (복원)
    decoded = tokenizer.decode(encoded['input_ids'][0])
    print(f"디코딩 결과: '{decoded}'")
    
    print("\n💡 특수 토큰 설명:")
    print("  [CLS]: 문장 시작 (Classification 토큰)")
    print("  [SEP]: 문장 끝 (Separator 토큰)")
    print("  [PAD]: 길이 맞춤용 패딩")
    print("  [UNK]: 모르는 단어")

demonstrate_tokenization(example_text)


🔍 토큰화 과정 시연
------------------------------
원본 문장: 'Natural language processing is amazing!'
토큰화 결과: ['natural', 'language', 'processing', 'is', 'amazing', '!']
토큰 ID: [3019, 2653, 6364, 2003, 6429, 999]
특수 토큰 포함: tensor([[ 101, 3019, 2653, 6364, 2003, 6429,  999,  102]])
디코딩 결과: '[CLS] natural language processing is amazing! [SEP]'

💡 특수 토큰 설명:
  [CLS]: 문장 시작 (Classification 토큰)
  [SEP]: 문장 끝 (Separator 토큰)
  [PAD]: 길이 맞춤용 패딩
  [UNK]: 모르는 단어


In [38]:
test_sentences = sample_texts[:3]
print("처리할 문장들:")
for i, sent in enumerate(test_sentences, 1):
    print(f"  {i}. {sent}")

처리할 문장들:
  1. Natural language processing is a fascinating field of artificial intelligence.
  2. Machine learning algorithms can process and analyze large amounts of text data.
  3. Deep learning models have revolutionized natural language understanding.


In [39]:
def get_bert_embeddings(texts, tokenizer, model):
    """
    BERT 임베딩 추출 함수
    
    Args:
        texts: 문장들의 리스트
        tokenizer: BERT 토크나이저
        model: BERT 모델
        
    Returns:
        numpy array: 각 문장의 임베딩 벡터들
    """
    print(f"\n🎯 BERT 임베딩 추출 함수 동작:")
    print("  1단계: 각 문장을 토큰화")
    print("  2단계: 토큰들을 BERT 모델에 입력")
    print("  3단계: [CLS] 토큰의 출력을 문장 임베딩으로 사용")
    
    embeddings = []
    
    # gradient 계산 비활성화 (추론 모드)
    with torch.no_grad():
        print(f"\n📝 {len(texts)}개 문장 처리 중...")
        
        for i, text in enumerate(texts):
            print(f"  처리 중: 문장 {i+1}/{len(texts)}")
            print(f"    원본: '{text}'")
            
            # 1단계: 토큰화 및 텐서 변환
            inputs = tokenizer(
                text, 
                return_tensors="pt",     # PyTorch 텐서로 반환
                padding=True,            # 길이 맞춤
                truncation=True,         # 최대 길이 초과시 자름
                max_length=512          # BERT 최대 입력 길이
            )
            
            print(f"    토큰 수: {inputs['input_ids'].shape[1]}개")
            
            # 2단계: BERT 모델 실행
            outputs = model(**inputs)
            
            # 3단계: [CLS] 토큰의 임베딩 추출
            # last_hidden_state: [배치크기, 시퀀스길이, 차원수]
            # [0]: 첫 번째 (유일한) 문장
            # [0]: 첫 번째 토큰 ([CLS])
            cls_embedding = outputs.last_hidden_state[0][0]
            
            print(f"    임베딩 차원: {cls_embedding.shape}")
            
            # numpy로 변환하여 저장
            embeddings.append(cls_embedding.numpy())
    
    return np.array(embeddings)

# 실제 임베딩 추출 (첫 3개 문장만 - 시간 절약)
print(f"\n🚀 실제 BERT 임베딩 추출 시작")
print("="*50)

bert_embeddings = get_bert_embeddings(test_sentences, tokenizer, model)

print(f"\n📊 BERT 임베딩 결과:")
print(f"  임베딩 배열 크기: {bert_embeddings.shape}")
print(f"  문장 수: {bert_embeddings.shape[0]}개")
print(f"  각 임베딩 차원: {bert_embeddings.shape[1]}차원")

print(f"\n🔍 첫 번째 문장 임베딩 상세:")
first_embedding = bert_embeddings[0]
print(f"  전체 차원: {len(first_embedding)}")
print(f"  처음 10차원: {first_embedding[:10]}")
print(f"  벡터 크기: {np.linalg.norm(first_embedding):.4f}")
print(f"  최대값: {np.max(first_embedding):.4f}")
print(f"  최소값: {np.min(first_embedding):.4f}")


🚀 실제 BERT 임베딩 추출 시작

🎯 BERT 임베딩 추출 함수 동작:
  1단계: 각 문장을 토큰화
  2단계: 토큰들을 BERT 모델에 입력
  3단계: [CLS] 토큰의 출력을 문장 임베딩으로 사용

📝 3개 문장 처리 중...
  처리 중: 문장 1/3
    원본: 'Natural language processing is a fascinating field of artificial intelligence.'
    토큰 수: 13개
    임베딩 차원: torch.Size([768])
  처리 중: 문장 2/3
    원본: 'Machine learning algorithms can process and analyze large amounts of text data.'
    토큰 수: 15개
    임베딩 차원: torch.Size([768])
  처리 중: 문장 3/3
    원본: 'Deep learning models have revolutionized natural language understanding.'
    토큰 수: 12개
    임베딩 차원: torch.Size([768])

📊 BERT 임베딩 결과:
  임베딩 배열 크기: (3, 768)
  문장 수: 3개
  각 임베딩 차원: 768차원

🔍 첫 번째 문장 임베딩 상세:
  전체 차원: 768
  처음 10차원: [-0.2655637  -0.12168896 -0.39667323 -0.0114592  -0.04621242 -0.12133733
  0.127688    0.3928875  -0.1328064  -0.50623184]
  벡터 크기: 13.2355
  최대값: 3.2072
  최소값: -7.4128


In [40]:
def cosine_similarity(a, b):
    """
    코사인 유사도 계산 함수
    
    Args:
        a, b: 두 벡터
        
    Returns:
        float: 코사인 유사도 (-1 ~ 1)
    """
    # 코사인 유사도 = A·B / (|A| × |B|)
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    
    return dot_product / (norm_a * norm_b)


# 문장 간 유사도 분석
print(f"\n📐 BERT 문장 유사도 분석")
print("-" * 40)

print("💡 코사인 유사도 해석:")
print("  1.0: 완전히 동일한 방향 (매우 유사)")
print("  0.0: 직교 (관련 없음)")
print("  -1.0: 완전히 반대 방향 (정반대)")

# 모든 문장 쌍의 유사도 계산
n_sentences = len(bert_embeddings)

print(f"\n📊 {n_sentences}개 문장 간 유사도 매트릭스:")
print("     ", end="")
for j in range(n_sentences):
    print(f"문장{j+1:2d}", end="  ")
print()

for i in range(n_sentences):
    print(f"문장{i+1:2d}", end=" ")
    for j in range(n_sentences):
        if i == j:
            sim = 1.0  # 자기 자신과는 완전 유사
        else:
            sim = cosine_similarity(bert_embeddings[i], bert_embeddings[j])
        print(f"{sim:6.3f}", end="  ")
    print()


📐 BERT 문장 유사도 분석
----------------------------------------
💡 코사인 유사도 해석:
  1.0: 완전히 동일한 방향 (매우 유사)
  0.0: 직교 (관련 없음)
  -1.0: 완전히 반대 방향 (정반대)

📊 3개 문장 간 유사도 매트릭스:
     문장 1  문장 2  문장 3  
문장 1  1.000   0.951   0.958  
문장 2  0.951   1.000   0.951  
문장 3  0.958   0.951   1.000  


In [41]:
# 가장 유사한/다른 문장 쌍 찾기
max_sim = -2.0
min_sim = 2.0
max_pair = None
min_pair = None

for i in range(n_sentences):
    for j in range(i+1, n_sentences):
        sim = cosine_similarity(bert_embeddings[i], bert_embeddings[j])
        if sim > max_sim:
            max_sim = sim
            max_pair = (i, j)
        if sim < min_sim:
            min_sim = sim
            min_pair = (i, j)

if max_pair:
    print(f"\n🏆 가장 유사한 문장 쌍:")
    print(f"  문장 {max_pair[0]+1}: '{test_sentences[max_pair[0]]}'")
    print(f"  문장 {max_pair[1]+1}: '{test_sentences[max_pair[1]]}'")
    print(f"  유사도: {max_sim:.4f}")

if min_pair:
    print(f"\n🔻 가장 다른 문장 쌍:")
    print(f"  문장 {min_pair[0]+1}: '{test_sentences[min_pair[0]]}'")
    print(f"  문장 {min_pair[1]+1}: '{test_sentences[min_pair[1]]}'")
    print(f"  유사도: {min_sim:.4f}")


🏆 가장 유사한 문장 쌍:
  문장 1: 'Natural language processing is a fascinating field of artificial intelligence.'
  문장 3: 'Deep learning models have revolutionized natural language understanding.'
  유사도: 0.9581

🔻 가장 다른 문장 쌍:
  문장 2: 'Machine learning algorithms can process and analyze large amounts of text data.'
  문장 3: 'Deep learning models have revolutionized natural language understanding.'
  유사도: 0.9507


#### BERT vs 전통적 방법들 비교

In [42]:
print("📊 임베딩 차원 비교:")
print(f"  TF-IDF: {len(tfidf_vectorizer.vocabulary_)}차원 (희소)")
if 'w2v_model' in globals():
    print(f"  Word2Vec: {w2v_model.wv.vector_size}차원 (밀집)")
if 'fasttext_model' in globals():
    print(f"  FastText: {fasttext_model.wv.vector_size}차원 (밀집)")
print(f"  BERT: {bert_embeddings.shape[1]}차원 (밀집, 문맥적)")

print(f"\n🎯 각 방법의 특징:")
print("  TF-IDF:")
print("    ✅ 빠름, 해석 용이")
print("    ❌ 단어 순서 무시, 의미 관계 부족")

print("  Word2Vec/FastText:")
print("    ✅ 의미적 유사성, 효율적")
print("    ❌ 문맥 의존성 부족")

print("  BERT:")
print("    ✅ 문맥 이해, 양방향성, 높은 성능")
print("    ❌ 계산 비용 높음, 복잡함")

print(f"\n💼 실제 사용 가이드:")
print("  🚀 빠른 프로토타입: TF-IDF")
print("  🎯 일반적 NLP: Word2Vec/FastText")
print("  🏆 최고 성능 필요: BERT/Transformers")

📊 임베딩 차원 비교:
  TF-IDF: 48차원 (희소)
  Word2Vec: 100차원 (밀집)
  FastText: 100차원 (밀집)
  BERT: 768차원 (밀집, 문맥적)

🎯 각 방법의 특징:
  TF-IDF:
    ✅ 빠름, 해석 용이
    ❌ 단어 순서 무시, 의미 관계 부족
  Word2Vec/FastText:
    ✅ 의미적 유사성, 효율적
    ❌ 문맥 의존성 부족
  BERT:
    ✅ 문맥 이해, 양방향성, 높은 성능
    ❌ 계산 비용 높음, 복잡함

💼 실제 사용 가이드:
  🚀 빠른 프로토타입: TF-IDF
  🎯 일반적 NLP: Word2Vec/FastText
  🏆 최고 성능 필요: BERT/Transformers


#### Transformers 실습 요약

✅ BERT 실습 성공!
- 처리된 문장 수: 3개
- 임베딩 차원: 768차원
- 핵심 특징: 문맥 의존적 임베딩

🎯 Transformers의 혁신:
  1. 문맥 이해 → 같은 단어도 상황에 따라 다른 의미
  2. 양방향성 → 앞뒤 문맥 모두 고려
  3. 전이 학습 → 대용량 사전 훈련 + 태스크별 미세 조정
  4. 범용성 → 하나의 모델로 여러 NLP 태스크 해결

-----

### **방법별 특징 요약**
<br>
BoW:<br>
- 장점: 간단, 빠름, 해석 가능<br>
- 단점: 희소 벡터, 단어 순서 무시, 의미 정보 부족<br>
- 사용처: 문서 분류, 텍스트 검색<br>
<br>
N-gram:<br>
- 장점: 지역적 단어 순서 고려, BoW 확장<br>
- 단점: 차원 폭발, 희소성 증가<br>
- 사용처: 언어 모델링, 구문 패턴 분석<br>
<br>
TF-IDF:<br>
- 장점: 단어 중요도 반영, 일반적인 단어 억제<br>
- 단점: 여전히 희소 벡터, 의미 정보 제한<br>
- 사용처: 정보 검색, 문서 유사도<br>
<br>
Word2Vec:<br>
- 장점: 밀집 벡터, 의미적 유사도 포착<br>
- 단점: OOV 문제, 정적 임베딩<br>
- 사용처: 단어 유사도, 의미적 추론<br>
<br>
GloVe:<br>
- 장점: 전역 통계 정보 활용, 좋은 성능<br>
- 단점: OOV 문제, 정적 임베딩<br>
- 사용처: 단어 임베딩, 전이 학습<br>
<br>
FastText:<br>
- 장점: OOV 처리 가능, 하위 단어 정보 활용<br>
- 단점: 메모리 사용량 많음<br>
- 사용처: 형태학이 복잡한 언어, OOV가 많은 상황<br>
<br>
Transformers:<br>
- 장점: 문맥적 임베딩, 최고 성능, 다양한 작업 가능<br>
- 단점: 계산 비용 높음, 메모리 사용량 많음<br>
- 사용처: 최신 NLP 작업 전반<br>
<br>
각 방법은 상황에 따라 장단점이 있으므로, 데이터 특성과 작업 목표에 맞게 선택하세요.